In [3]:
import os
import argparse
import pprint
from datetime import date
import _init_paths

import numpy as np
import random
import warnings
import torch
import torch.nn as nn
from config import config, update_config
import create_logger
from lib import metric_clip as metric


from core.model_prompt import model
from dataset.iterator_factory import get_dataiter
from networks.get_symbol import get_symbol
from lib import criterions


torch.backends.cudnn.enabled = False

In [2]:
update_config('./exps/clip.yaml')
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

if config.seed:
    random.seed(config.seed)
    np.random.seed(config.seed)
    torch.manual_seed(config.seed)
    torch.cuda.manual_seed(config.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    warnings.warn('You have chosen to seed training. '
                    'This will turn on the CUDNN deterministic setting, '
                    'which can slow down your training considerably! '
                    'You may see unexpected behavior when restarting '
                    'from checkpoints.')

data_loader, data_iter = get_dataiter('data_name', config.data.root, config.data.set, config.net.name, config,
                                        config.data.max_rul, config.data.seq_len, 'clip')

/home/qchenax/astar/rul_prompt/config.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  exp_config = edict(yaml.load(f))
/tmp/ipykernel_69172/3993644186.py:11: UserWarning: You have chosen to seed training. This will turn on the CUDNN deterministic setting, which can slow down your training considerably! You may see unexpected behavior when restarting from checkpoints.
  warnings.warn('You have chosen to seed training. '


Sensor::test data (23) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (24) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (20) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (21) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (19) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (28) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (29) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (19) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (20) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (28) less than seq_len (30)!
Sensor::Use first data to pad!
Sensor::test data (29) less than seq_len (30)!
Sensor::Use first data to pad!
Setting::test data (23) less than seq_len (30)!
Setting::Use first data to pad!
Setting::test data (24) less than seq_len (30)!
Setting::Use f

In [33]:
for i_batch, dats in enumerate(data_loader):
    intypes = ((dats[2].cpu().numpy())*125).astype(np.int_)[:,0]
    intypes_uni = set(intypes.flatten())
    fulltypes = set(np.arange(126).flatten())
    complement = list(fulltypes - intypes_uni)
    samples = random.sample(complement, len(complement))
    fulllabels = np.concatenate((intypes, np.array(samples)), axis=0)
    fulllabels = torch.tensor(fulllabels[:, np.newaxis]/125)
    print(fulllabels)
    break

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [0.5280],
        [1.0000],
        [0.7920],
        [1.0000],
        [0.2000],
        [0.7280],
        [0.2560],
        [1.0000],
        [1.0000],
        [0.8320],
        [1.0000],
        [0.0080],
        [1.0000],
        [0.4400],
        [0.2160],
        [1.0000],
        [0.0720],
        [1.0000],
        [0.5520],
        [0.2000],
        [1.0000],
        [0.5440],
        [0.8000],
        [0.4320],
        [1.0000],
        [1.0000],
        [0.0000],
        [1.0000],
        [1.0000],
        [0.2160],
        [0.9120],
        [0.3280],
        [1.0000],
        [1.0000],
        [1.0000],
        [0.2960],
        [1.0000],
        [0.2000],
        [0.5680],
        [0.2640],
        [0.9680],
        [0.1200],
        [0.4480],
        [0.4720],
        [1.0000],
        [1.0000],
        [1.0000],
        [0.9360],
        [0.7440],
        [0.1200],
        [1.0000],
        [0

In [5]:
a = nn.Parameter(torch.ones([]) * np.log(10))

b = torch.tensor([1, 2, 3, 4])
a*b

tensor([2.3026, 4.6052, 6.9078, 9.2103], grad_fn=<MulBackward0>)